In [2]:
from helper import *

path='../reports/corpus.xlsx'

description = get_column_values(path, 'descriptions_measures', 'A')


targets = get_column_values(path, 'descriptions_measures', 'B')

similarities_df = similarities_df(docs=description,group=targets,encoding_method='tfidf')

similarities_df.head()

,"Provide a report that displays the number of accounts with running settlements for the Earth portfolio, their expected monthly payments at the end of the examined month and their actual monthly payments per DCA, for September 2020.","Provide a report that displays the number of accounts not in running settlements for the Earth portfolio, their expected monthly payments at the end of the examined month and their actual monthly payments per DCA, for September 2020.","Provide the number of customers with active and running settlements of types Settlement, Preappoved, Resch to Sett, Out of Mandate Had Settlement and null on the Earth portfolio, the corresponding number of applications, the initial settlement amount, discount amount and future instalments per asset class and DCA, for September 2020.","Provide the distribution of the number of active and running settlements of types Settlement, Preappoved, Resch to Sett, Out of Mandate Had Settlement and null on the Earth portfolio based on their duration on the following bins: up to 6 months, 7 to 12 months, 13 to 36 months, 37 to 72 months, 72 to 108 months and more than 109 months. The analysis should be performane per asset class and DCA, for September 2020.","Produce a report that shows the bucket roll rates.\nOn the rows of the report display the application bucket of the previous month for the following categories: 1, 2, 3, 4, 5, 6, 7-12, 13+, null\nOn the columns of the report display the application bucket of the current month for the same bucket categories.\nThe report should measure the number of active and running applications for each pair of the aforementioned previous/current bucket categories for Mirror portfolio, for September 2020.","Create a report that shows the number of settlement applications submitted during the month, their approved amount and their entry principal and balance, for September 2020 per DCA and application type. The report should be produced on Earth portfolio.","Create a report that shows the number of pending settlement applications (in status Review, Working, Quality Control, For approval and Approved) submitted up to the end of the previous month, their approved amount, the average pending days and their entry principal and balance, for September 2020 per DCA and application type. The report should be produced on Earth portfolio.","Create a report that shows the number of settlement applications approved during the month, their approved amount, the written off balance, the average days to approval, their average and median duration and their entry principal and balance, for September 2020 per DCA and application type. The report should be produced on Earth portfolio.","Create a report that shows the number of settlement applications rejected during the month, their approved amount, the average days to rejection, their entry principal and entry balance, for September 2020 per DCA and application type. The report should be produced on Earth portfolio."
Application status at the end of the preceding observation period,0.357232,0.354629,0.247133,0.204734,0.385322,0.208833,0.314276,0.231332,0.240294
Status of the last submitted application within the observation period.,0.250025,0.248203,0.249181,0.206430,0.388516,0.274619,0.321964,0.233249,0.242285
Application type in the observation period,0.148040,0.198788,0.120273,0.107592,0.252628,0.219130,0.254805,0.221240,0.238728
Last rejection date of all applications within the observation perion,0.143531,0.142485,0.189671,0.127450,0.225444,0.144132,0.158791,0.142733,0.204240
Total payments amount so far,0.136085,0.135093,0.056934,0.000000,0.000000,0.038475,0.027405,0.028336,0.035359


In [2]:
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

path = '../reports/corpus.xlsx'

targets = get_column_values(path, 'descriptions_measures', 'B')




similarities_dict = get_similarity_scores(text=Q1,documents=targets,encoding_method='bow')

top_10 = get_first_n_keys(similarities_dict, 10)


top_10



['Status of the last submitted application within the observation period.',
 'Application status at the end of the preceding observation period',
 'Application type in the observation period',
 'Amount paid within the observation period',
 'Balance written off',
 'Application bucket in the previous observation period',
 'Last rejection date of all applications within the observation perion',
 'Approved amount',
 'Approved duration',
 'Days to approval']

In [ ]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')

d = get_similarities_for_values(descriptions,targets,'bow',10)

df = pd.DataFrame.from_dict(d)

df


In [ ]:
from helper import *



path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')

similarities_dict = get_similarities_for_values(descriptions,targets,'sbert',10)

df = pd.DataFrame.from_dict(similarities_dict)

#save_df_to_excel(df,'../reports/sbert.xlsx','sbert')

In [ ]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
n = 10

for model in ['bow','tfidf','word2vec','sbert']:
    print(f'==================== {model} ====================')
    for i in range(9):
        d = results_to_targets(descriptions, targets, model, n, path,i)
        print(f'Candidate templates for Q{i+1}')
        print(d)

In [ ]:
from helper import *

path='../reports/corpus.xlsx'
model = 'bow'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
n = 10

#d = results_to_targets(descriptions, targets, model, n, path)
print('==================== BOW ====================')
for i in range(9):
    d = results_to_targets(descriptions, targets, model, n, path,i)
    print(f'Candidate templates for Q{i+1}')
    print(d)

In [ ]:
from helper import *

path='../reports/corpus.xlsx'
model = 'word2vec'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
n = 10

#d = results_to_targets(descriptions, targets, model, n, path)
print('==================== W2V ====================')
for i in range(9):
    d = results_to_targets(descriptions, targets, model, n, path,i)
    print(f'Candidate templates for Q{i+1}')
    print(d)

In [ ]:
from helper import *

path='../reports/corpus.xlsx'
model = 'sbert'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
n = 10

#d = results_to_targets(descriptions, targets, model, n, path)
print('==================== SBERT ====================')
for i in range(9):
    d = results_to_targets(descriptions, targets, model, n, path,i)
    print(f'Candidate templates for Q{i+1}')
    print(d)

In [1]:
from test_save_load import *
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

path = '../reports/corpus.xlsx'

targets = get_column_values(path, 'descriptions_measures', 'B')




similarities_dict = get_similarity_scores_new(text=Q1,documents=targets,encoding_method='tfidf',version='tfidf_model.joblib')

top_10 = get_first_n_keys(similarities_dict, 10)


top_10

c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Balance written off',
 'Status of the last submitted application within the observation period.',
 'Application status at the end of the preceding observation period',
 'Approved duration',
 'Approved amount',
 'Application type in the observation period',
 'Days to approval',
 'Amount paid within the observation period',
 'Application bucket in the previous observation period',
 'Last rejection date of all applications within the observation perion']